### The code is for benchmarking the war and peace journalism dataset with different language models

In [ ]:
### Change the variable model_name to whichever model you wish to use. Ensure to make other relevant changes (some models may not require any change). 
### The required changes have to be made inside LanguageModel class, change self.model = '' to appropriate model initialization, and in intializing the tokenizer.

In [ ]:
import pandas as pd
import numpy as np
import sys
import torch

from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, ModernBertModel, T5Tokenizer, T5EncoderModel

from transformers import get_scheduler

import torch.nn as nn
from torch.optim import AdamW

from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
class LanguageModel(nn.Module):
    
    def __init__(self, attr, siamese_training, num_classes = 2):
        
        super(LanguageModel, self).__init__()
        torch.manual_seed(0)
        # self.model = T5EncoderModel.from_pretrained(model_name)
        # self.model = ModernBertModel.from_pretrained(model_name)
        # self.model = AutoModelForMaskedLM.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        # self.model.load_state_dict(torch.load(f'models/siamese/con_headline_roberta_lab_{attr}.pth'))
        
        
        # self.attention_vector = nn.Linear(self.model.config.hidden_size, 1, bias=False)
        self.dropout = nn.Dropout(p = 0.15)
        self.classifier = nn.Linear(self.model.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_masks):
        
        # x = self.model(input_ids = input_ids, attention_mask = attention_masks).last_hidden_state.mean(dim=1)
        emb = self.model(input_ids = input_ids, attention_mask = attention_masks)
        # hs = emb.hidden_states
        # emb = torch.stack([x.mean(axis = 1) for x in hs]).mean(axis = 0)
        # emb = emb.last_hidden_state.mean(dim=1)
        
        # attn_scores = self.attention_vector(emb.last_hidden_state).squeeze(-1)  # Shape: (batch, seq_len)
        # attn_weights = F.softmax(attn_scores, dim=1)  # Normalize across tokens
        # emb = (attn_weights.unsqueeze(-1) * emb.last_hidden_state).sum(dim=1) 
        
        # emb, _ = emb.last_hidden_state.max(dim=1)
        
        emb = emb.last_hidden_state[:, 0, :]
        
        # last_hidden_state
        # print(self.model.config.hidden_size)
        # .pooler_output
        # print("Here ", x.shape)
        # sys.exit()
        # .pooler_output
        x = self.dropout(emb)
        x = self.classifier(x)
        
        return x
    
def get_dataset(sent, labels):
    
    tokens = tokenizer(sent, return_tensors="pt", padding=True, truncation=True)
    # tokens = tokenizer(sent, return_tensors="pt", padding='max_length', truncation=True)
    labels = torch.tensor(labels)
    data = TensorDataset(tokens['input_ids'], tokens['attention_mask'], labels)

    return DataLoader(data, batch_size = 64)

def get_perf(dataloader):
    
    total_val_f1 = 0
    for batch in dataloader:
        
        # emb = torch.tensor([emb])
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        # print(tok)
        labels = labels.float()
        
        with torch.no_grad():
            logits = model(input_ids, attention_mask)
            # logits = model(input_ids)
            
        preds = torch.argmax(logits, dim=-1)
        
        total_val_f1 += f1_score(labels.cpu(), preds.cpu(), average = 'weighted')
        
    avg_val_f1 = total_val_f1 / len(dataloader)
    
    return avg_val_f1
            

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def mask_entities(sentences: pd.Series) -> pd.Series:
    def mask(sentence):
        doc = nlp(sentence)
        masked = sentence
        for ent in doc.ents:
            masked = masked.replace(ent.text, "<mask>")
        return masked
    return sentences.apply(mask)

def get_data(attr):
    
    df = pd.read_csv(f'data/Label_{attr}.csv')
    # print(df.columns)
    
    df['headline'] = mask_entities(df['headline'])
    
    return df

In [ ]:
# class_mapping = dict({'1':'emotive', '4a': 'effects', '4b': 'source', '4c': 'context', '3a': 'villain', '3b': 'victim'})

In [ ]:
torch.manual_seed(0)
# model_name = 'bert-base-cased'
model_name = 'roberta-base'
# model_name = 'daodao/ConflictBERT'
# model_name = 'snowood1/ConfliBERT-scr-cased'
# model_name = 'facebook/bart-base'
# model_name = 'google/gemma-2-9b'
# model_name = 'studio-ousia/luke-base'
# model_name = 'studio-ousia/mluke-base'
# model_name = 'allenai/OLMoE-1B-7B-0924'
# model_name = 'answerdotai/ModernBERT-base'
# model_name = 'meta-llama/Llama-3.2-1B'
# model_name = 't5-base'

# from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# tokenizer = T5Tokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
device = torch.device('cuda:0')

# siamese_training = 'verb_subspace'
# 'head_cont_and_verb_subspace'
# verb_subspace
# 'simple_contrastive'

# for siamese_training in ['verb_subspace', 'head_cont_and_verb_subspace', 'simple_contrastive']:

siamese_training = None
# attr = '1'
# for attr in ['4a']:
for attr in ['emotive', 'effects', 'source', 'context', 'villain', 'victim']:
    # ['1', '4a', '4b']:
    # 
    # for attr in ['1', '3a', '3b']:
    data = get_data(attr)
    X_train, X_test, train_label, test_label = train_test_split(data.headline, data.label, test_size=0.2, stratify = data.label, random_state = 42)

    train_dataloader = get_dataset(X_train.tolist(), train_label.tolist())
    test_dataloader = get_dataset(X_test.tolist(), test_label.tolist())

    model = LanguageModel(attr, siamese_training, num_classes = len(set(train_label.tolist())))
    # model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=len(set(train_label.tolist())))
    # model.config.pad_token_id = tokenizer.pad_token_id
    # model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(train_label.tolist())))
    # model.to(device)
    model = nn.DataParallel(model).to(device)
    
    'Model Loaded!'

    optimizer = AdamW(model.parameters(), lr = 5e-5, eps = 1e-8, weight_decay=0.05)
    loss_fn = nn.CrossEntropyLoss()

    epochs = 50
    num_training_steps = epochs * 8
    lr_scheduler = get_scheduler(
                name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
            )
    loss_values = []
    train_perf, test_perf = [], []

    progress_bar = tqdm(range(num_training_steps))

    for epoch in range(epochs):

        model.train()
        batch_loss_values = []

        np.random.seed(epoch)
        torch.manual_seed(epoch)
        torch.cuda.manual_seed_all(epoch)
        # random.seed(epoch)
        random_ids = np.random.randint(len(train_dataloader.dataset), size=512)
        temp_train_dataloader = train_dataloader.dataset[random_ids]
        # dataset = TensorDataset(temp_train_dataloader[0], temp_train_dataloader[1])
        dataset = TensorDataset(temp_train_dataloader[0], temp_train_dataloader[1], temp_train_dataloader[2])
        sampler = RandomSampler(dataset)
        dataloader = DataLoader(dataset, batch_size=64, sampler = sampler)

        for batch in dataloader:

            model.zero_grad()

            # emb = torch.tensor([emb])
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            # print(tok)
            # labels = labels.float()

            # outputs = bert(input_ids, attention_mask)

            logits = model(input_ids, attention_mask)
            # logits = model(input_ids)
            # print(logits)
            # sys.exit()

            # logits = logits.squeeze(-1)
            loss = loss_fn(logits, labels)

            reg_lambda = 0.00001
            # l1_loss = sum([p.abs().sum() for p in model.parameters()])
            l2_loss = sum([(p ** 2).sum() for p in model.parameters()])
            loss = reg_lambda * l2_loss + loss

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            progress_bar.update(1)

            # print("Here ", loss) 

            batch_loss_values.append(loss.item())

        model.eval()

        tr_perf = get_perf(train_dataloader)
        tst_perf = get_perf(test_dataloader)

        train_perf.append(tr_perf)
        test_perf.append(tst_perf)

        avg_batch_loss = sum(batch_loss_values)/len(batch_loss_values)

        # print(f'Epoch {epoch} completed! Training Loss: {avg_batch_loss}. Training F1: {tr_perf} and Testing F1: {tst_perf}')
    temp_df = pd.DataFrame({'train': train_perf, 'test': test_perf})
    temp_df.to_csv(f'results/benchmark/roberta-base/Label_{attr}_cls.csv')
    torch.save(model, f"models/benchmark/roberta-base/Label_{attr}_cls.pth")

    # torch.save(model.state_dict(), f"models/roberta/Label_{attr}.pth")

    # temp_df.to_csv(f'results/siamese/{siamese_training}/roberta_Label_{attr}.csv')
    # torch.save(model, f"models/contrastive_models/{siamese_training}/roberta_Label_{attr}.pth")